In [1325]:
%config IPCompleter.greedy=True
from queue import *
import sys

In [1326]:
class IndependentSet():
    def __init__(self, g):
        self.G = g
        self.Forest = Graph(g.order())

    def I(self):
        return self.Forest.edges()

In [1327]:
# https://medium.com/100-days-of-algorithms/day-41-union-find-d0027148376d
def find(data, i):
    if i != data[i]:
        data[i] = find(data, data[i])
    return data[i]

def union(data, i, j):
    pi, pj = find(data, i), find(data, j)
    if pi != pj:
        data[pi] = pj

In [1328]:
def rootTree(self, root, treenumber, I):
    d = DiGraph()
    q = Queue()
    q.put(root)
    levelordernumber = 0
    
    while not q.empty():
        v = q.get()
        levelordernumber += 1
        I.Levelordernumbers[v] = levelordernumber
        I.Visited[v] = True
        I.Treenumbers[v] = treenumber
        
        for n in self.neighbor_iterator(v):
            if not I.Visited[n]:
                d.add_edge(v,n)
                q.put(n)
                
    return d
        
Graph.rootTree = rootTree

In [1312]:
def c_graphic((a,b), I):
    if I.Treenumbers[a] != I.Treenumbers[b]:
        return None
    
    global disjointsets
    cycle = []
    tree = I.Trees[I.Treenumbers[a]]
    while a != b:
        node = a if I.Levelordernumbers[a] > I.Levelordernumbers[b] else b
        parent = tree.neighbors_in(node)[0]
        if find(disjointsets, parent) != find(disjointsets, node):
            cycle.append((parent, node))
            union(disjointsets, parent, node)
        if I.Levelordernumbers[a] > I.Levelordernumbers[b]:
            a = parent
        else:
            b = parent
    
    return cycle

In [1313]:
def start_graphic(Is):
    global disjointsets
    disjointsets = [i for i in range(Is[0].Forest.order())]
    for I in Is:
        I.Treenumbers = [0] * I.Forest.order()
        I.Trees = list()
        I.Levelordernumbers = [0] * I.Forest.order()
        I.Visited = [False] * I.Forest.order()
        components = I.Forest.connected_components_subgraphs()
        for idx, c in enumerate(components):
            rootedTree = c.rootTree(c.vertices()[0], idx, I)
            I.Trees.append(rootedTree)

In [1314]:
def cyclic_scanning(e, k, labels, Is, c):
    q = Queue()
    q.put(e)
    while not q.empty():
        e = q.get()
        (_,i) = labels[e]
        i = (i + 1) % k
        cycle = c(e, Is[i])
        if cycle == None:
            return ((e,i), labels)
        for f in cycle:
            labels[f] = (e, i)
            q.put(f)
    return (None, labels)

In [1315]:
def k_forest(g, k, start, c):
    Is = [IndependentSet(g) for i in range(k)]
    
    removedEdges = []
    for v in filter(lambda v: g.degree(v) <= k, g.vertices()):
        for idx, e in enumerate(g.edges_incident([v], labels = False)):
            removedEdges.append(e + (idx,))
        g.delete_vertex(v)
    
    start(Is)
    
    label = None
    for e in g.edges(labels = False):
        labels = {}
        labels[e] = (e,-1)
        if label != None:
            start(Is)
        (label, labels) = cyclic_scanning(e, k, labels, Is, c)
        if label != None:
            (f,i) = label
            while i != -1:
                Is[i].Forest.add_edge(f + (i,))
                if f != e:
                    i = (i - 1) % k
                    Is[i].Forest.delete_edge(f + (i,))
                (f,i) = labels[f]
    
    for (a, b, idx) in removedEdges:
        Is[idx].Forest.add_edge((a, b, idx))
        g.add_edge((a, b))  
        
    return Is

In [1316]:
def top_clump(g, k, start, c):
    L = []
    B = []
    Is = k_forest(g, k, start, c)
    for i in range(k):
        B += Is[i].I()
    for e in B:
        start(Is)
        labels = {}
        labels[e[:2]] = (e,-1)
        (label, labels) = cyclic_scanning(e[:2], k, labels, Is, c)
        if label == None:
            L.append(e)
    return L

In [1317]:
def c_dual((a,b), I):
    global disjointsets
    cycle = []
    mincut = I.G.edge_cut(a, b, value_only=False, use_edge_labels = True)[1]
    mincut = [(x,y) for (x,y,_) in mincut if (x,y) in I.J]
    if mincut == []:
        return None
    for (a,b) in mincut:
        if find(disjointsets, a) != find(disjointsets, b):
            cycle.append((a, b))
            union(disjointsets, a, b)
    return cycle

In [1318]:
def start_dual(Is):
    global disjointsets
    disjointsets = [i for i in range(Is[0].Forest.order())]
    m = Matroid(Is[0].G)
    for I in Is:
        span = m.closure([e[:2] for e in I.Forest.edges()])
        #I.J = list(set(sp).difference(set(map(lambda x: x[:2], I.Forest.edges()))))
        I.J = [(x,y) for (x,y) in span if (x,y) not in map(lambda z: z[:2], I.Forest.edges())]

In [1319]:
def top_clump_dual(g, k, start, c, start_dual, c_dual):
    B = []
    A = []
    Is = k_forest(g, k, start, c)
    start(Is)
    for i in range(k):
        B += Is[i].I()
        for e in Is[i].I():
            for j in range(i + 1, i + k):
                j = j % k
                if c(e[:2], Is[j]) == None:
                    A.append(e[:2])
                    break
    start_dual(Is)
    labels = {}
    for e in A: labels[e] = (e,-1)
    for e in A: (label, labels) = cyclic_scanning(e, k, labels, Is, c_dual)
    return filter(lambda x: x[:2] not in labels.keys(), B)

In [ ]:
graph = Graph(8) #Graph({0: [1], 1: [2,3], 3: [4,5]})
graph.add_edge(0,1)
graph.add_edge(0,2)
graph.add_edge(1,2)
graph.add_edge(2,3)
graph.add_edge(1,3)
graph.add_edge(3,6)
graph.add_edge(3,4)
graph.add_edge(3,5)
graph.add_edge(3,7)
graph.add_edge(4,5)
graph.add_edge(5,7)
graph.add_edge(4,6)
graph.add_edge(5,6)
graph.add_edge(7,6)
pos = {0:[0.0, 0.0], 1:[1.0, 1.0], 2:[1.0, -1.0], 3:[2.0, 0.0], 4:[2.8, 0.1], 5:[3.2, -0.3], 6:[4.0, 1.0], 7:[4.0, -1.0]}


#graph = Graph(4) #Graph({0: [1], 1: [2,3], 3: [4,5]})
#graph.add_edge(0,1)
#graph.add_edge(0,2)
#graph.add_edge(0,3)
#graph.add_edge(2,3)
#graph.add_edge(1,3)
#pos = {0:[0.0, 0.0], 1:[1.0, 1.0], 2:[1.0, -1.0], 3:[2.0, 0.0]}

#graph = graphs.CycleGraph(9)
#pos = graph.layout().copy()

#graph = graphs.PoussinGraph()
#pos = graph.layout()

#graph = graphs.FruchtGraph()
#pos = graph.layout()

#graph = graphs.GoldnerHararyGraph()
#graph.delete_edge((1,3,None))
#pos = graph.layout()

#graph = graphs.HerschelGraph()
#pos = graph.layout()

#graph = graphs.WienerArayaGraph()
#pos = graph.layout()

#graph = graphs.CompleteGraph(5)
#pos = graph.layout()

graph.weighted(True)

print (len(graph.edges()))
p = graph.plot(vertex_labels=True, graph_border=True, pos=pos)
#p.show()
disjointsets = []
Is = k_forest(graph, 2, start_graphic, c_graphic)
l1 = Is[0].I()
l2 = Is[1].I()
l = l1 + l2
print(len(l))
res1 = set(map(lambda x: x[:2], graph.edges()))
res2 = set(map(lambda x: x[:2], l))
res = res1.difference(res2)
r = map(lambda x: x + (-1,), list(res))
output = Graph(4)
output.add_edges(r + l)
pp = output.plot(vertex_labels=True, graph_border=True, color_by_label={0:(1,0,0), 1:(0,1,0)}, pos=pos)
#pp.show()

In [ ]:
L = top_clump(graph, 2, start_graphic, c_graphic)
print(len(L))
res1 = set(map(lambda x: x[:2], graph.edges()))
res2 = set(map(lambda x: x[:2], L))
res = res1.difference(res2)
r = map(lambda x: x + (-1,), list(res))
gg = Graph({})
gg.add_edges(r + L)
p = gg.plot(vertex_labels=True, graph_border=True, color_by_label={0:(1,0,0), 1:(0,1,0)}, pos=pos)
#p.show()

In [ ]:
L = top_clump_dual(graph, 2, start_graphic, c_graphic, start_dual, c_dual)
print(len(L))
res1 = set(map(lambda x: x[:2], graph.edges()))
res2 = set(map(lambda x: x[:2], L))
res = res1.difference(res2)
r = map(lambda x: x + (-1,), list(res))
ggg = Graph({})
ggg.add_edges(r + L)
p = ggg.plot(vertex_labels=True, graph_border=True, color_by_label={0:(1,0,0), 1:(0,1,0)}, pos=pos)
#p.show()


In [1323]:
g = Graph(2)
g.add_edge(0,1)
g.add_edge(0,1)
#y = map(lambda x: x[:2], g.edges())
#z = map(lambda x: x + (-2,), list(y))
#print (z)
#print (g.edges())
#g.plot()
#print(graph.edges())
#x = graph.edge_cut(0, 1, value_only=False, use_edge_labels = True)[1]
#print(x)
#w = [(a,b) for (a,b,_) in x if (a,b) not in [(0,1)]]
#print(w)